# Catboost 최적화

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('./data/train.csv').drop(columns=['UID'])

In [3]:
df.info()   # non-null: 이상치 없음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   주거 형태              10000 non-null  object 
 1   연간 소득              10000 non-null  float64
 2   현재 직장 근속 연수        10000 non-null  object 
 3   체납 세금 압류 횟수        10000 non-null  float64
 4   개설된 신용계좌 수         10000 non-null  int64  
 5   신용 거래 연수           10000 non-null  float64
 6   최대 신용한도            10000 non-null  float64
 7   신용 문제 발생 횟수        10000 non-null  int64  
 8   마지막 연체 이후 경과 개월 수  10000 non-null  int64  
 9   개인 파산 횟수           10000 non-null  int64  
 10  대출 목적              10000 non-null  object 
 11  대출 상환 기간           10000 non-null  object 
 12  현재 대출 잔액           10000 non-null  float64
 13  현재 미상환 신용액         10000 non-null  float64
 14  월 상환 부채액           10000 non-null  float64
 15  신용 점수              10000 non-null  int64  
 16  채무 불이행 여부          1000

In [4]:
Y = df['채무 불이행 여부']
X = df.drop('채무 불이행 여부', axis=1)

print(X)

                주거 형태      연간 소득 현재 직장 근속 연수  체납 세금 압류 횟수  개설된 신용계좌 수  \
0                  자가  1941337.5      10년 이상          0.0           9   
1                  월세  1979505.0      10년 이상          0.0           5   
2                  월세  1356381.0          4년          0.0          12   
3                  월세  1049017.5          6년          0.0          15   
4                  월세  4320217.5          2년          0.0          11   
...               ...        ...         ...          ...         ...   
9995  주택 담보 대출 (거주 중)  1339473.0      10년 이상          0.0           9   
9996  주택 담보 대출 (거주 중)  2297230.5          2년          0.0          11   
9997  주택 담보 대출 (거주 중)  1221523.5      10년 이상          0.0           9   
9998               자가  3343584.0      10년 이상          0.0          10   
9999  주택 담보 대출 (거주 중)  2175133.5          5년          0.0           5   

      신용 거래 연수   최대 신용한도  신용 문제 발생 횟수  마지막 연체 이후 경과 개월 수  개인 파산 횟수   대출 목적  \
0         13.4  400597.5            0        

In [5]:
# 원 핫 인코딩
categories = ['주거 형태', '현재 직장 근속 연수', '대출 목적', '대출 상환 기간']


X = pd.get_dummies(X, columns = categories, dtype=int) 

X

,연간 소득,체납 세금 압류 횟수,개설된 신용계좌 수,신용 거래 연수,최대 신용한도,신용 문제 발생 횟수,마지막 연체 이후 경과 개월 수,개인 파산 횟수,현재 대출 잔액,현재 미상환 신용액,...,대출 목적_소규모 사업 자금,대출 목적_여행 자금,대출 목적_의료비,대출 목적_이사 비용,대출 목적_자동차 구매,대출 목적_주택 개보수,대출 목적_주택 구매,대출 목적_휴가 비용,대출 상환 기간_단기 상환,대출 상환 기간_장기 상환
0,1941337.5,0.0,9,13.4,400597.5,0,24,1,390903.0,225457.5,...,0,0,0,0,0,0,0,0,1,0
1,1979505.0,0.0,5,15.1,360679.5,0,11,0,1002184.5,64749.0,...,0,0,0,0,0,0,0,0,1,0
2,1356381.0,0.0,12,18.8,491770.5,1,74,3,227775.0,487644.0,...,0,0,0,0,0,0,0,0,1,0
3,1049017.5,0.0,15,14.8,411546.0,1,22,1,251383.5,413211.0,...,0,0,0,0,0,0,0,0,1,0
4,4320217.5,0.0,11,26.1,895288.5,0,32,0,1163176.5,78991.5,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1339473.0,0.0,9,18.7,319027.5,0,68,0,126216.0,177028.5,...,0,0,0,0,0,0,0,0,1,0
9996,2297230.5,0.0,11,28.3,399799.5,0,7,0,371907.0,347449.5,...,0,0,0,0,0,1,0,0,0,1
9997,1221523.5,0.0,9,30.1,823305.0,0,14,0,869736.0,176905.5,...,0,0,0,0,0,0,0,0,0,1
9998,3343584.0,0.0,10,20.3,724314.0,0,25,0,443008.5,139294.5,...,0,0,0,0,0,0,0,0,1,0


In [6]:
# 로그
cl = ['현재 대출 잔액', '현재 미상환 신용액', '월 상환 부채액']

for i in cl:
    X[i] = X[i].apply(lambda x: np.log1p(x))

X

,연간 소득,체납 세금 압류 횟수,개설된 신용계좌 수,신용 거래 연수,최대 신용한도,신용 문제 발생 횟수,마지막 연체 이후 경과 개월 수,개인 파산 횟수,현재 대출 잔액,현재 미상환 신용액,...,대출 목적_소규모 사업 자금,대출 목적_여행 자금,대출 목적_의료비,대출 목적_이사 비용,대출 목적_자동차 구매,대출 목적_주택 개보수,대출 목적_주택 구매,대출 목적_휴가 비용,대출 상환 기간_단기 상환,대출 상환 기간_장기 상환
0,1941337.5,0.0,9,13.4,400597.5,0,24,1,12.876217,12.325891,...,0,0,0,0,0,0,0,0,1,0
1,1979505.0,0.0,5,15.1,360679.5,0,11,0,13.817694,11.078289,...,0,0,0,0,0,0,0,0,1,0
2,1356381.0,0.0,12,18.8,491770.5,1,74,3,12.336118,13.097343,...,0,0,0,0,0,0,0,0,1,0
3,1049017.5,0.0,15,14.8,411546.0,1,22,1,12.434739,12.931716,...,0,0,0,0,0,0,0,0,1,0
4,4320217.5,0.0,11,26.1,895288.5,0,32,0,13.966666,11.277108,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1339473.0,0.0,9,18.7,319027.5,0,68,0,11.745758,12.084072,...,0,0,0,0,0,0,0,0,1,0
9996,2297230.5,0.0,11,28.3,399799.5,0,7,0,12.826402,12.758377,...,0,0,0,0,0,1,0,0,0,1
9997,1221523.5,0.0,9,30.1,823305.0,0,14,0,13.675946,12.083377,...,0,0,0,0,0,0,0,0,0,1
9998,3343584.0,0.0,10,20.3,724314.0,0,25,0,13.001346,11.844353,...,0,0,0,0,0,0,0,0,1,0


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42)

In [12]:
from catboost import CatBoostClassifier
import optuna
from sklearn.metrics import accuracy_score

# Optuna 최적화 함수 정의
def optimize_catboost(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 100, 1000),  # 학습 반복 횟수
        "depth": trial.suggest_int("depth", 3, 10),  # 트리 깊이
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),  # 학습률
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1.0, 10.0),  # L2 정규화
        "border_count": trial.suggest_int("border_count", 32, 255),  # 이진화 기준 개수
        "random_strength": trial.suggest_float("random_strength", 0.1, 2.0),  # 랜덤성 조절
    }

    # 모델 생성 및 학습
    model = CatBoostClassifier(**params, verbose=0, random_state=42)
    model.fit(X_train, y_train)
    
    # 예측 및 성능 평가
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)  # 정확도 최적화

# Optuna 스터디 생성 및 최적화 실행
study_catboost = optuna.create_study(direction="maximize")
study_catboost.optimize(optimize_catboost, n_trials=100)  # 50번 탐색

# 최적의 하이퍼파라미터 출력
best_params_catboost = study_catboost.best_params
print("Best CatBoost Hyperparameters:", best_params_catboost)

# 최적의 하이퍼파라미터로 모델 학습
best_catboost = CatBoostClassifier(**best_params_catboost, verbose=0, random_state=42)
best_catboost.fit(X_train, y_train)

# 최적 모델 평가
y_pred_best = best_catboost.predict(X_test)
final_accuracy = accuracy_score(y_test, y_pred_best)
print("Final Optimized CatBoost Accuracy:", final_accuracy)


[I 2025-03-20 12:36:19,985] A new study created in memory with name: no-name-1290be99-f65c-4ea9-a465-7971d3818907
[I 2025-03-20 12:36:25,617] Trial 0 finished with value: 0.7 and parameters: {'iterations': 482, 'depth': 10, 'learning_rate': 0.2957920030594448, 'l2_leaf_reg': 3.3090425819440217, 'border_count': 201, 'random_strength': 1.7811037828645948}. Best is trial 0 with value: 0.7.
[I 2025-03-20 12:36:26,478] Trial 1 finished with value: 0.7152 and parameters: {'iterations': 488, 'depth': 3, 'learning_rate': 0.12641840907211224, 'l2_leaf_reg': 9.41377934820463, 'border_count': 174, 'random_strength': 0.11887912485300885}. Best is trial 1 with value: 0.7152.
[I 2025-03-20 12:36:29,842] Trial 2 finished with value: 0.6976 and parameters: {'iterations': 303, 'depth': 10, 'learning_rate': 0.22391493678690189, 'l2_leaf_reg': 7.120072999507957, 'border_count': 200, 'random_strength': 0.9103999855347658}. Best is trial 1 with value: 0.7152.
[I 2025-03-20 12:36:30,280] Trial 3 finished wi

Best CatBoost Hyperparameters: {'iterations': 610, 'depth': 3, 'learning_rate': 0.08202873455802175, 'l2_leaf_reg': 8.38496952312003, 'border_count': 145, 'random_strength': 1.548870459171905}
Final Optimized CatBoost Accuracy: 0.7272
